In [44]:
import requests as rq
from bs4 import BeautifulSoup
import ujson
import pandas as pd
from tqdm.auto import tqdm
import random
from time import sleep
from os import environ
import traceback

In [24]:
def is_valid(json, key):
    if not isinstance(json, dict):
        return None
    if key in json.keys():
        return key
    ans = None
    for json_key in json.keys():
        r = is_valid(json[json_key], key)
        if r is None:
            continue
        else :
            ans = "{}.{}".format(json_key, r)
    return ans

def get_abs_path(json, key):
    path = is_valid(json, key)
    print(path)

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36',
    'Accept-Language': 'en,en-US;q=0,5',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,/;q=0.8'
}

json_content_list = []
loop_counter = 0
page_number = 1

url = f'https://www.saatchiart.com/paintings/realism/architecture?hitsPerPage=100$page={page_number}&styles=photorealism'

In [40]:
# Get architecture images
with open("views_likes_architecture.txt", 'a') as f:
    for page_number in tqdm(range(1, 101)):
        url = f'https://www.saatchiart.com/paintings/realism/architecture?hitsPerPage=100&page={page_number}&styles=photorealism'
        page_html = rq.get(url, headers=headers)
        soup = BeautifulSoup(page_html.content)
        page_json_dict = ujson.loads(soup.find('script', {'id': '__NEXT_DATA__'}).contents[0])

        for artwork in page_json_dict['props']['pageProps']['initialState']['algolia']['data']['hits']['results']:
            artwork_id = artwork['artworkId']
            artwork_url = artwork['artworkUrl']
            if artwork_url[0] != 'h':
                artwork_url = 'https://www.saatchiart.com/' + artwork_url
            image_url = artwork['imageUrl']
            f.write(
                str(artwork_id) + ',' +
                str(artwork_url) + ',' +
                str(image_url) + '\n')

In [41]:
# Get cities images
with open("views_likes_cities.txt", 'a') as f:
    for page_number in tqdm(range(1, 82)):
        url = f'https://www.saatchiart.com/paintings/realism/cities?hitsPerPage=100&page={page_number}&styles=photorealism'
        page_html = rq.get(url, headers=headers)
        soup = BeautifulSoup(page_html.content)
        page_json_dict = ujson.loads(soup.find('script', {'id': '__NEXT_DATA__'}).contents[0])

        for artwork in page_json_dict['props']['pageProps']['initialState']['algolia']['data']['hits']['results']:
            artwork_id = artwork['artworkId']
            artwork_url = artwork['artworkUrl']
            if artwork_url[0] != 'h':
                artwork_url = 'https://www.saatchiart.com/' + artwork_url
            image_url = artwork['imageUrl']
            f.write(
                str(artwork_id) + ',' +
                str(artwork_url) + ',' +
                str(image_url) + '\n')

In [46]:
df_architecture = pd.read_csv('views_likes_architecture.txt', header=None)
df_architecture.columns = ['artwork_id', 'artwork_url', 'image_url']

In [ ]:
likes_list = []
views_list = []
for artwork_id, artwork_url in tqdm(zip(
        list(df_architecture['artwork_id']),
        list(df_architecture['artwork_url']))):
    page_html = rq.get(artwork_url, headers=headers)
    soup = BeautifulSoup(page_html.content, features='html.parser')
    if soup is not None:
        j = soup.find('script', {'id': '__NEXT_DATA__'}).contents[0]
        j = ujson.loads(j)['props']['pageProps']['initialState']['page']['data']['artwork']
        views = j['views']
        likes = j['likes']
    else:
        views = -999
        likes = -999
    likes_list.append(likes)
    views_list.append(views)

df_architecture['likes'] = likes_list
df_architecture['views'] = views_list
df_architecture.head()

In [ ]:

df_cities = pd.read_csv('views_likes_architecture.txt', header=None)
df_cities.columns = ['artwork_id', 'artwork_url', 'image_url']

likes_list = []
views_list = []
for artwork_id, artwork_url in tqdm(zip(
        list(df_cities['artwork_id']),
        list(df_cities['artwork_url']))):
    page_html = rq.get(artwork_url, headers=headers)
    soup = BeautifulSoup(page_html.content, features='html.parser')
    j = soup.find('script', {'id': '__NEXT_DATA__'}).contents[0]
    j = ujson.loads(j)['props']['pageProps']['initialState']['page']['data']['artwork']
    views = j['views']
    likes = j['likes']
    likes_list.append(likes)
    views_list.append(views)

df_cities['likes'] = likes_list
df_cities['views'] = views_list
df_cities.head()

In [56]:
len(views_list)
pass
pass

736